<a href="https://colab.research.google.com/github/Private-Detective/CloudStream/blob/main/XL8R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src='https://storage.googleapis.com/pr-newsroom-wp/1/2018/11/Spotify_Logo_RGB_Green.png' height="39"/>

In [ ]:

from IPython.display import clear_output

# Check if spotdl is installed
SPOTDL = !(command -v "spotdl")
if not SPOTDL:
    !pip install spotdl
    clear_output()

# Add search query option
search_query = ""  # @param {type:"string"}
link = ""  # @param {type:"string"}
output = "/content/{title} - {artists}"  # kept the original output format

options = [
    "--format mp3",
    "--bitrate 128k",  # 128k as default bitrate
    "--lyrics musixmatch",  # Musixmatch as default lyrics provider
    "--sponsor-block",
    "--generate-lrc",
    "--print-errors",
    f"--output \"{output}\""
]

# Add link to spotdl command if provided
if link:
    !spotdl download {link} {" ".join(options)}
else:
    !spotdl download "{search_query}" {" ".join(options)}

try:
    # No need for eval or exec, just execute the command directly
    pass
except FileNotFoundError:
    print(f"Error: spotdl command not found. Please install spotdl.")
except Exception as e:
    print(f"Error: {e}")

# Add a success message
print("Download complete!")

# <img src='https://upload.wikimedia.org/wikipedia/commons/5/5f/FFmpeg_Logo_new.svg' height="34"/>

### *Encoder with loop*

In [ ]:
import os
import time

input_file = "" #@param {type:"string"}
output_file = "" #@param {type:"string"}
arguments = "" #@param {type:"string"}

while True:
    try:
        os.system(f"ffmpeg -y -i {input_file} {arguments} {output_file}")
        break
    except Exception as e:
        print(f"An error occurred: {e}")
        time.sleep(1)

### *Subtitle Encoder*

In [ ]:
input1 = "" #@param {type:"string"}
input2 = "" #@param {type:"string"}
filename = "" #@param {type:"string"}
!ffmpeg -hide_banner -i "$input1" -i "$input2" -c copy -map_metadata -1 -map_chapters -1 drive/MyDrive/"$filename".mkv

### *Video Splitter Tool*

In [ ]:
Input = "" #@param {type:"string"}
!ffmpeg -hide_banner -y -i "$Input" -c copy -map_metadata -1 -map_chapters -1 -segment_time 00:01:00 -f segment "drive/MyDrive/DriveFire/Segments/file%02d.mkv"

In [ ]:
import os

input_dir = "drive/MyDrive/DriveFire/Segments"
ffpb = 'drive/MyDrive/ffpb.py'
output_dir = "drive/MyDrive"

for filename in sorted(os.listdir(input_dir)):
    if filename.endswith("mkv"):
        input_file = os.path.join(input_dir, filename)
        output_file = os.path.join(output_dir, filename)
        !chmod 755 $ffpb
        print(f"Processing file: {filename}")
        !$ffpb -hide_banner -y -i $input_file -c:v libx265 -map_metadata -1 -map_chapters -1 -c:a copy $output_file
        print(f"File processed: {filename}")

In [ ]:
with open("list.txt", "w") as f:
    for i in range(0,10):
        f.write(f"file drive/MyDrive/file0{i}.mkv\n")

In [ ]:
!printf "file '%s'\n" *.mkv | sort -V > list.txt

In [ ]:
!ffmpeg -hide_banner -y -f concat -safe 0 -i list.txt -c copy -map 0 -map_metadata -1 -map_chapters -1 f.mkv

In [ ]:
!cat list.txt

file drive/MyDrive/file00.mkv
file drive/MyDrive/file01.mkv
file drive/MyDrive/file02.mkv
file drive/MyDrive/file03.mkv
file drive/MyDrive/file04.mkv
file drive/MyDrive/file05.mkv
file drive/MyDrive/file06.mkv
file drive/MyDrive/file07.mkv
file drive/MyDrive/file08.mkv
file drive/MyDrive/file09.mkv


### *Multi-Image Generator*

In [ ]:
input_file = "" #@param {type:"string"}
seek_time = "" #@param {type:"string"}
num_of_frames = 10 #@param {type:"integer", default: 10}
increment = 1 #@param {type:"integer", default: 1}

import os
import glob
from IPython.display import Image, display, Markdown
import datetime

for filename in glob.glob('*.jpg'):
    os.remove(filename)

if ':' in seek_time:
    seek_time_dt = datetime.datetime.strptime(seek_time, "%H:%M:%S")
else:
    seek_time_dt = datetime.datetime(1900, 1, 1) + datetime.timedelta(seconds=int(seek_time))

for i in range(num_of_frames):
    timestamp_dt = seek_time_dt + datetime.timedelta(seconds=i*increment)
    timestamp = timestamp_dt.strftime("%H:%M:%S")
    command = f"ffmpeg -y -loglevel error -ss {timestamp} -i \"{input_file}\" -frames:v 1 /content/{timestamp}.jpg"
    os.system(command)

for filename in sorted(glob.glob('*.jpg'), key=lambda x: int(x[:-4].replace(':', ''))):
    display(Image(filename))
    display(Markdown(f"***{filename[:-4]}***"))

for filename in glob.glob('*.jpg'):
    os.remove(filename)

### *Image Generator*

In [ ]:
import os
import subprocess

input = "" #@param {type:"string"}
seek_normal = "" #@param {type:"string"}
filter = "" #@param {type:"string"}
first_frame = False #@param {type:"boolean"}
last_frame = False #@param {type:"boolean"}

if not os.path.exists(input):
  print("Error: Input file not found")
  exit()

if first_frame:
  !ffmpeg -loglevel error -y -i "{input}" -vframes 1 -q 0 first.jpg
if last_frame:
  duration = int(float(subprocess.check_output(f"ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 -i \"{input}\"", shell=True).decode("utf-8").strip()))
  !ffmpeg -loglevel error -y -ss {duration} -i "{input}" -vframes 1 -q 0 last.jpg
if seek_normal:
  if filter:
    !ffmpeg -loglevel error -y -ss {seek_normal} -i "{input}" -vf "{filter}" -vframes 1 -q 0 normal.jpg
  else:
    !ffmpeg -loglevel error -y -ss {seek_normal} -i "{input}" -vframes 1 -q 0 normal.jpg

import glob
from IPython.display import Image, display
jpg_files = glob.glob('*.jpg')
for file in jpg_files:
  display(Image(file))
!rm -rf *jpg

### *Trimmer*

In [ ]:
input = "" #@param {type:"string"}
output = "" #@param {type:"string"}
start = "" #@param {type:"string"}
end = "" #@param {type:"string"}
re_encode = False #@param {type:"boolean"}
from datetime import datetime

x = datetime.strptime(start, "%H:%M:%S")
y = datetime.strptime(end, "%H:%M:%S")
dur = y - x

if not re_encode:
  !ffmpeg -y -loglevel error -i "$input" -ss "$start" -t "$dur" -avoid_negative_ts 1 -c copy "$output".mkv
else:
  !ffmpeg -y -loglevel error -ss "$start" -i "$input" -to "$end" -crf 0 -c:a copy "$output".mkv

### *MediaInfo*

In [ ]:
import subprocess
import json
import os

input_file = "" #@param {type:"string"}

if not input_file:
    print("Error: No input file provided.")
else:
    file_size_mb = os.path.getsize(input_file) / (1024 * 1024)
    print(f"File size: {file_size_mb / 1024 if file_size_mb > 1024 else file_size_mb:.2f} {'GB' if file_size_mb > 1024 else 'MB'}")

    try:
        ffprobe_output = subprocess.check_output(f"ffprobe -v error -show_streams -show_format -print_format json \"{input_file}\"", shell=True).decode("utf-8")
        media_info = json.loads(ffprobe_output)

        print("Media Information:")
        for stream in media_info['streams']:
            print(f"  {stream['codec_type'].capitalize()}:")
            print(f"    Codec: {stream['codec_name']}")
            if stream['codec_type'] == 'video':
                print(f"    Resolution: {stream['width']}x{stream['height']}")
            elif stream['codec_type'] == 'audio':
                print(f"    Channels: {stream['channel_layout']}")

        duration_output = subprocess.check_output(f"ffprobe -v error -show_entries format=duration -sexagesimal -of default=noprint_wrappers=1:nokey=1 -i \"{input_file}\"", shell=True).decode("utf-8").strip()
        print(f"Duration: {duration_output}")

    except subprocess.CalledProcessError:
        print("Error: Unable to get media information using ffprobe.")

# <img src='https://biplobsd.github.io/RLabClone/img/title_youtube-dl.png' height="38"/>

In [ ]:
YTDL=!(command -v "yt-dlp")
if not YTDL:
  !curl -s -L https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp -o /usr/local/bin/yt-dlp
  !chmod a+rx /usr/local/bin/yt-dlp
URL = "" #@param {type:"string"}
quality = '0' #@param ["bv[ext=mp4]+ba[ext=m4a]","0"] {allow-input: true}
output = "%(title)s.%(ext)s" #@param ["%(title)s.%(ext)s"] {allow-input: true}
!yt-dlp --no-warnings -q --progress -f {quality} -o "$output" "$URL"

# <font size=5> ✦ *Colab Still Alive*

In [ ]:
some_str = ' ' * 5120000000000

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
import IPython
from IPython.display import clear_output
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Clicked on the connect button");
     btn.click()
     }

   btn = document.getElementById('connect')
   if (btn != null){
     console.log("Clicked on the reconnect button");
     btn.click()
     }
  }

setInterval(ClickConnect,60000)
'''))

clear_output()

# <font size=6> Scratchpad

In [ ]:
#@markdown <font size=4>***Demucs***
from IPython.display import clear_output
input = "" #@param{type:"string"}
DEMUCS=!(command -v "demucs")
if not DEMUCS:
  !python3 -m pip install -U demucs
  clear_output()
!demucs "$input"

In [ ]:
#@markdown <font size=4>***Auto Sub***

input_file = "" #@param {type:"string"}
output_file = "" #@param {type:"string"}
device = "cuda" #@param ["cuda","cpu"]
compute_type = "float16" #@param ["float16","int8"]

from IPython.display import clear_output
!pip install faster-whisper
clear_output()

from faster_whisper import WhisperModel

model_size = "large-v2"
model = WhisperModel(model_size, device, compute_type=compute_type)

segments, _ = model.transcribe(input_file, beam_size=50, language="ko", task="translate", word_timestamps=True, vad_filter=True)

with open(output_file, "w") as f:
    for i, segment in enumerate(segments):
        print(f"Transcribing segment {i+1}/{len(segments)}")
        for word in segment.words:
            f.write("[%.2fs -> %.2fs] %s\n" % (word.start, word.end, word.word))

In [ ]:
#@markdown <font size=4>***Google Translator***
!pip install googletrans
import googletrans
from googletrans import Translator  # Import the Translator class from the googletrans library

file = "" #@param {type:"string"}
translator = Translator()  # Create a Translator object
with open(file, 'r') as f:  # Open the file in read mode
    text = f.read()  # Read the contents of the file
translated = translator.translate(text, src='ko', dest='en')  # Translate the text
print(translated.text)  # Print the translated text

In [ ]:
file= "" #@param{type:"string"}
from google.colab import files
files.download(file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>